# Try out GQCP: the Ghent Quantum Chemistry Package

Welcome to the try-it-out Notebook that we've set up to guide you through your first developer steps with GQCP.

## Setup

In order to set up this Notebook for Binder, just run the following cells.

In [ ]:
!pip install plotly plotly-express pandas 

In [ ]:
import os
os.environ["LIBINT_DATA_PATH"] = "/srv/conda/envs/notebook/share/libint/2.4.2/basis"

## Import Python modules

In [ ]:
import gqcpy
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go

## Generating PES diagrams for RHF and FCI

In this example, we'll compare the potential energy surface (PES) for H$_2$ with two well-established quantum chemical methods.

RHF (restricted Hartree-Fock) is an approximate method, and we'll examine its well-known behavior in the dissociated regime. On the other hand, FCI (full configuration interaction) is exact (within a given basis set).

We'll start off by creating a set of molecules that corresponds to the points on the PES we would like to calculate.

In [ ]:
# Initialize an array of distances and set up an empty molecule list.
distances = np.arange(0.5, 5, 0.05)
molecules = []

# Create a fixed hydrogen nucleus (with atomic charge '1').
# The other nuclei will be placed at one of the distances.
left = gqcpy.Nucleus(1, 0, 0, 0)

for i in distances:
    right = gqcpy.Nucleus(1, 0, 0, i)
    
    # Form a molecule from the nuclei, and add them to the molecule list.
    molecule = gqcpy.Molecule([left, right], charge=0)
    molecules.append(molecule)

### Setting up the RHF calculations

Let's make a small function that returns a list of RHF energies when provided our list of molecules as an argument.

In [ ]:
def calculate_RHF_energies(molecules, basis_set='STO-3G'):
    
    energies = []
    for molecule in molecules:
        
        # Let's start by creating the (second-quantized) molecular Hamiltonian in the basis of the AOs.
        spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, basis_set)  # Generate the spin-orbital basis for the molecule in the given basis set. The suffix '_d' is needed for real-valued calculations.
        hamiltonian = gqcpy.RSQHamiltonian_d.Molecular(spin_orbital_basis, molecule)  # Create a second-quantized molecular in the previously set up spin-orbital basis. Note that the Hamiltonian is expressed in the non-orthogonal basis of the AOs.
        
        # For RHF SCF (self-consistent field), we need a solver and an objective. We'll also have to set up a calculation environment.
        N = molecule.numberOfElectrons()
        S = spin_orbital_basis.quantizeOverlapOperator()  # This is the overlap matrix of the AOs.
        environment = gqcpy.RHFSCFEnvironment_d.WithCoreGuess(N, hamiltonian, S)  # We've initialized the RHF SCF environment with an initial guess being the (generalized) eigenvectors of the core Hamiltonian.
        solver = gqcpy.RHFSCFSolver_d.Plain()  # For this example, we'll use a plain RHF SCF solver, but we offer many more kinds of SCF solvers. Check out the documentation on our web page for more information.
        objective = gqcpy.DiagonalRHFFockMatrixObjective_d(hamiltonian)  # Using this objective will make sure that we obtain the canonical RHF MOs at the end of the calculation.

        # Perform the RHF SCF calculation by an `.optimize` call.
        qc_structure = gqcpy.RHF_d.optimize(objective, solver, environment)  # `.optimize` calls return a quantum chemical structure (`QCStructure`), which is a general list of (energy,solution)-pairs for each state. RHF SCF only finds the ground state, so the `qc_structure` will only have one member.

        # Calculate the total energy of the molecule.
        nuclear_repulsion = gqcpy.Operator.NuclearRepulsion(molecule).value()
        total_energy = qc_structure.groundStateEnergy() + nuclear_repulsion
        energies.append(total_energy)
    
    return energies

Note that the paradigm `environment - solver - optimize - qc_structure` is generally applied to any of the quantum chemical methods that GQCP offers. More information can be found on the web page.

### Setting up the FCI calculations

Similarly to the RHF case, let's set up an FCI function that will calculate all the FCI energies of an input list of molecules.

In [ ]:
def calculate_FCI_energies(molecules, basis_set='STO-3G'):

    energies = []    
    for molecule in molecules:
        
        # We'll start by setting up the molecular Hamiltonian quantized in an orthonormal spin-orbital basis. We could use the RHF MOs, but since we're doing full CI, we might as well just use the Löwdin basis (S^{-1/2}).
        spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, basis_set)  # Generate the initial spin-orbital basis for the molecule in the given basis set.
        spin_orbital_basis.lowdinOrthonormalize()
        hamiltonian = gqcpy.RSQHamiltonian_d.Molecular(spin_orbital_basis, molecule)

        # For an FCI calculation, we have to set up the complete ONV (occupation number vector) basis.
        K = spin_orbital_basis.numberOfSpatialOrbitals()
        N_P = molecule.numberOfElectronPairs()
        onv_basis = gqcpy.SpinResolvedONVBasis(K, N_P, N_P)

        # Set up a solver, environment, and call the `.optimize` function to get a quantum chemical structure.
        solver = gqcpy.EigenproblemSolver.Dense()  # For this example, we'll just use a dense eigensolver. Check out the documentation on our web page for more information on other kinds of solvers.
        environment = gqcpy.CIEnvironment.Dense(hamiltonian, onv_basis)
        qc_structure = gqcpy.CI(onv_basis).optimize(solver, environment)
    
        # Calculate the total energy.
        nuclear_repulsion = gqcpy.Operator.NuclearRepulsion(molecule).value()
        total_energy = qc_structure.groundStateEnergy() + nuclear_repulsion

        energies.append(total_energy)
    
    return energies

## Visualizing the results

We'll use [plotly](https://plotly.com/graphing-libraries/) for the visualization, which creates amazing and interactive figures. Check them out by hovering your curser over the graph!

In [ ]:
RHF_energies = calculate_RHF_energies(molecules)
FCI_energies = calculate_FCI_energies(molecules)

In [ ]:
figure = go.Figure()
figure.add_trace(go.Scatter(name='RHF', x=distances, y=RHF_energies))
figure.add_trace(go.Scatter(name='FCI', x=distances, y=FCI_energies))

figure.update_xaxes(title_text='Internuclear distance (Bohr)',
                    ticks="inside",
                    nticks = 5)
figure.update_yaxes(title_text='Energy (Hartree)',
                    ticks="inside")

figure.update_layout(legend=
    dict(yanchor="top",
         y=0.99,
         xanchor="right",
         x=0.99)
)